## Exercise - Pricing Swaptions

In [1]:
import warnings

import numpy as np
import pandas as pd
from scipy.stats import norm

warnings.filterwarnings("ignore")

In [2]:
# Load the necessary swaption volatility & cap curves data
swaption_vol = pd.read_excel('../data/swaption_vol_data_2025-06-30.xlsx', sheet_name = 0)
rate_data = pd.read_excel('../data/cap_curves_2025-06-30.xlsx', sheet_name = 0)

# Display the loaded datasets
display(swaption_vol.style.set_caption("Swaption Volatility Data"))
display(rate_data.style.set_caption("Cap Curves Data"))

,reference,instrument,model,date,expiration,tenor,-200,-100,-50,-25,0,25,50,100,200
0,SOFR,swaption,black,2025-06-30 00:00:00,1,1,72.250000,46.870000,39.100000,36.000000,33.390000,31.300000,29.760000,28.170000,28.660000
1,SOFR,swaption,black,2025-06-30 00:00:00,1,2,65.780000,44.400000,37.970000,35.460000,33.390000,31.750000,30.530000,29.190000,29.300000
2,SOFR,swaption,black,2025-06-30 00:00:00,1,3,57.870000,40.610000,35.560000,33.650000,32.110000,30.920000,30.060000,29.140000,29.290000
3,SOFR,swaption,black,2025-06-30 00:00:00,1,4,54.405000,38.565000,33.925000,32.195000,30.830000,29.805000,29.095000,28.430000,28.885000
4,SOFR,swaption,black,2025-06-30 00:00:00,1,5,50.940000,36.520000,32.290000,30.740000,29.550000,28.690000,28.130000,27.720000,28.480000


,tenor,swap rates,spot rates,discounts,forwards,flat vols,fwd vols
0,0.250000,0.042353,0.042353,0.989523,nan,nan,nan
1,0.500000,0.040859,0.040852,0.979883,0.039351,0.156842,0.156842
2,0.750000,0.039391,0.039372,0.971043,0.036414,0.180709,0.201708
3,1.000000,0.038115,0.038083,0.962807,0.034217,0.204576,0.240464
4,1.250000,0.036704,0.036653,0.955417,0.030938,0.242127,0.328341
5,1.500000,0.035655,0.035590,0.948239,0.030280,0.268642,0.336521
6,1.750000,0.034942,0.034868,0.941054,0.030542,0.285885,0.336809
7,2.000000,0.034453,0.034374,0.933835,0.030919,0.295615,0.328654
8,2.250000,0.034000,0.033916,0.926827,0.030248,0.299596,0.312413
9,2.500000,0.033750,0.033665,0.919605,0.031414,0.299589,0.296022


## 1. Pricing the Swaption

### 1.1 Calculate the Forward Swap Rate

To price a swaption, we first need to determine the forward swap rate $F$ for the underlying forward-starting swap. For a swap starting at time $T_n$ (the option expiration) and ending at time $T_N$ (where $T_N = T_n + \text{tenor}$), the forward swap rate is the rate that sets the present value of the swap to zero at inception.

Under the assumption of quarterly payment frequency ($\Delta = 0.25$), the forward swap rate is computed as:

$$F = \frac{Z(0, T_n) - Z(0, T_N)}{A(0, T_n, T_N)}$$

where $Z(0, t)$ is the discount factor for maturity $t$, and $A(0, T_n, T_N)$ is the forward annuity factor (or PV01), defined as:

$$A(0, T_n, T_N) = \sum_{i=n+1}^{N} \Delta \cdot Z(0, t_i)$$

We will interpolate the discount factors from the `rate_data` term structure.

In [3]:
# ---------------------------------------------------------
# Helper Functions for Curve Interpolation and Swap Math
# ---------------------------------------------------------

def get_discount(t, curve_df):
    """
    Extracts the discount factor for a given tenor t. 
    Uses linear interpolation for robustness against floating-point mismatches.
    """
    return np.interp(t, curve_df['tenor'].values, curve_df['discounts'].values)

def calc_fwd_swap_metrics(T_n, tenor, curve_df, freq=0.25):
    """
    Calculates the Forward Swap Rate and the Annuity Factor (PV01).
    
    Parameters:
    - T_n: Option expiration / Swap start time (in years)
    - tenor: Underlying swap tenor (in years)
    - curve_df: DataFrame containing the rate curve (must have 'tenor' and 'discounts')
    - freq: Payment frequency (0.25 for quarterly)
    
    Returns:
    - F: Forward Swap Rate
    - A: Annuity Factor
    """
    T_N = T_n + tenor
    
    # Generate payment dates (t_1 to t_N)
    payment_dates = np.arange(T_n + freq, T_N + freq / 2, freq)
    
    # Extract discount factors for all payment dates
    discounts = np.array([get_discount(t, curve_df) for t in payment_dates])
    
    # Calculate Annuity factor A
    A = np.sum(discounts) * freq
    
    # Extract bounding discount factors
    Z_n = get_discount(T_n, curve_df)
    Z_N = get_discount(T_N, curve_df)
    
    # Calculate Forward Swap Rate F
    F = (Z_n - Z_N) / A
    
    return F, A

# ---------------------------------------------------------
# Calculate for the 1y x 4y Swaption
# ---------------------------------------------------------
T_n_base = 1.0
tenor_base = 4.0

F_base, A_base = calc_fwd_swap_metrics(T_n_base, tenor_base, rate_data)

# Display the calculated metrics
swap_metrics_df = pd.DataFrame({
    "Metric": ["Forward Swap Rate (F)", "Annuity Factor (A)"],
    "Value": [F_base, A_base],
    "Display": [f"{F_base:.6f} ({F_base * 100:.4f}%)", f"{A_base:.6f}"]
})

swap_metrics_df = swap_metrics_df[["Metric", "Display"]]
swap_metrics_df.style.set_caption("1y x 4y Forward Swap Metrics")

,Metric,Display
0,Forward Swap Rate (F),0.032698 (3.2698%)
1,Annuity Factor (A),3.603238


### 1.2 Price the Swaptions across the Volatility Skew

We price the European payer swaption using Black's model (Black-76), taking the annuity factor as the numeraire. For a payer swaption with strike $K$, expiration $T$, and implied Black volatility $\sigma$, the price (as a percentage of notional) is:

$$V_{\text{payer}} = A \cdot \left[ F \cdot \Phi(d_1) - K \cdot \Phi(d_2) \right]$$

where:
$$d_1 = \frac{\ln(F/K) + \frac{1}{2}\sigma^2 T}{\sigma \sqrt{T}}, \quad d_2 = d_1 - \sigma \sqrt{T}$$

We will extract the specific row for the 1y x 4y swaption from `swaption_vol`, iterate over the provided strike offsets, calculate the absolute strikes, and compute the theoretical prices.

In [4]:
def black_payer_swaption(F, K, T, sigma, A):
    """
    Prices a European Payer Swaption using Black's Model.
    """
    if F <= 0 or K <= 0 or T <= 0 or sigma <= 0:
        return 0.0
    
    d1 = (np.log(F / K) + 0.5 * sigma**2 * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    price = A * (F * norm.cdf(d1) - K * norm.cdf(d2))
    return price

# ---------------------------------------------------------
# Pricing across the Volatility Skew
# ---------------------------------------------------------

# Isolate the 1x4 swaption volatility row
vol_row = swaption_vol[(swaption_vol['expiration'] == 1) & (swaption_vol['tenor'] == 4)].iloc[0]

# Define the skew columns we want to evaluate
skew_cols = [-200, -100, -50, -25, 0, 25, 50, 100, 200]
offsets_bps = [-200, -100, -50, -25, 0, 25, 50, 100, 200]

results_1x4 = []

# Iterate over offsets to price the skew
for col, offset in zip(skew_cols, offsets_bps):
    # Volatilities are given in percentages in the dataframe, so divide by 100
    sigma = vol_row[col] / 100.0 
    
    # Calculate absolute strike K (offset is in basis points)
    K = F_base + (offset / 10000.0)
    
    # Price the swaption
    price_pv = black_payer_swaption(F_base, K, T_n_base, sigma, A_base)
    
    results_1x4.append({
        "Offset (bps)": offset,
        "Strike (K)": K,
        "Implied Vol (σ)": sigma,
        "Price (PV / Notional)": price_pv,
        "Price (bps)": price_pv * 10000
    })

df_skew_prices = pd.DataFrame(results_1x4)

display(
    df_skew_prices.style
    .format({
        "Strike (K)": "{:.6f}",
        "Implied Vol (σ)": "{:.4f}",
        "Price (PV / Notional)": "{:.8f}",
        "Price (bps)": "{:.3f}"
    })
    .set_caption("1y x 4y Payer Swaption Prices Across the Skew")
)

# Store the ATM Volatility for Section 1.3
atm_vol = df_skew_prices.loc[df_skew_prices["Offset (bps)"] == 0, "Implied Vol (σ)"].values[0]

,Offset (bps),Strike (K),Implied Vol (σ),Price (PV / Notional),Price (bps)
0,-200,0.012698,0.5441,0.07271096,727.110
1,-100,0.022698,0.3856,0.03948049,394.805
2,-50,0.027698,0.3392,0.02536235,253.623
3,-25,0.030198,0.3220,0.01943130,194.313
4,0,0.032698,0.3083,0.01443366,144.337
5,25,0.035198,0.2980,0.01042391,104.239
6,50,0.037698,0.2909,0.00736560,73.656
7,100,0.042698,0.2843,0.00355738,35.574
8,200,0.052698,0.2888,0.00087983,8.798


## 1.3 Alternate Swaptions (Tenor and Expiration Sensitivity)

To observe how Black's formula responds to variations in the time-to-expiration ($T$) and the underlying tenor length (which drives the annuity factor $A$), we price the following alternate ATM swaptions:
* **3mo x 4yr** ($T=0.25$, Tenor=4)
* **2yr x 4yr** ($T=2.0$, Tenor=4)
* **1yr x 3yr** ($T=1.0$, Tenor=3)

We hold the implied volatility constant at the 1y x 4y ATM level to strictly isolate the structural impact of $T$ and $A$ on the optionality time value.

In [5]:
# ---------------------------------------------------------
# Pricing Alternate ATM Swaptions
# ---------------------------------------------------------

alternate_structures = [
    {"Name": "1yr x 4yr (Base)", "T_n": 1.0,  "Tenor": 4.0},
    {"Name": "3mo x 4yr",        "T_n": 0.25, "Tenor": 4.0},
    {"Name": "2yr x 4yr",        "T_n": 2.0,  "Tenor": 4.0},
    {"Name": "1yr x 3yr",        "T_n": 1.0,  "Tenor": 3.0}
]

alt_results = []

for struct in alternate_structures:
    T_n = struct["T_n"]
    tenor = struct["Tenor"]
    
    # 1. Calculate new F and A
    F, A = calc_fwd_swap_metrics(T_n, tenor, rate_data)
    
    # 2. Strike is ATM
    K = F
    
    # 3. Price using the constant ATM Vol from 1x4
    price_pv = black_payer_swaption(F, K, T_n, atm_vol, A)
    
    alt_results.append({
        "Swaption": struct["Name"],
        "Expiration (T)": T_n,
        "Tenor": tenor,
        "Forward ATM (F)": F,
        "Annuity (A)": A,
        "Price (PV / Notional)": price_pv,
        "Price (bps)": price_pv * 10000
    })

df_alt_swaptions = pd.DataFrame(alt_results)

# Calculate Relative Differences compared to the base 1x4 swaption
base_price_pv = df_alt_swaptions.loc[df_alt_swaptions["Swaption"] == "1yr x 4yr (Base)", "Price (PV / Notional)"].values[0]

df_alt_swaptions["Diff to Base (bps)"] = (df_alt_swaptions["Price (PV / Notional)"] - base_price_pv) * 10000
df_alt_swaptions["Diff to Base (%)"] = (df_alt_swaptions["Price (PV / Notional)"] / base_price_pv - 1) * 100

display(
    df_alt_swaptions.style
    .format({
        "Expiration (T)": "{:.2f}",
        "Tenor": "{:.2f}",
        "Forward ATM (F)": "{:.6f}",
        "Annuity (A)": "{:.4f}",
        "Price (PV / Notional)": "{:.8f}",
        "Price (bps)": "{:.3f}",
        "Diff to Base (bps)": "{:+.3f}",
        "Diff to Base (%)": "{:+.2f}%"
    })
    .set_caption("ATM Swaption Comparison (Constant Volatility)")
)

,Swaption,Expiration (T),Tenor,Forward ATM (F),Annuity (A),Price (PV / Notional),Price (bps),Diff to Base (bps),Diff to Base (%)
0,1yr x 4yr (Base),1.00,4.00,0.032698,3.6032,0.01443366,144.337,+0.000,+0.00%
1,3mo x 4yr,0.25,4.00,0.032998,3.6922,0.00748498,74.850,-69.487,-48.14%
2,2yr x 4yr,2.00,4.00,0.034257,3.4845,0.02059942,205.994,+61.658,+42.72%
3,1yr x 3yr,1.00,3.00,0.031918,2.7468,0.01074067,107.407,-36.930,-25.59%
